In [2]:
import geopandas as gpd
import json
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.geometry import MultiPolygon
from shapely.geometry import shape
from scipy.spatial import Delaunay
import pandas as pd
import numpy as np

In [200]:
shp_path = 'nv_2020_final/nv_2020_final.shp'
shapefile = gpd.read_file(shp_path)
shapefile = shapefile.to_crs(epsg=4326)
shapefile['WHT'] = shapefile['WHT'].fillna(0)
shapefile['STATEFP20'] = shapefile['STATEFP20'].fillna(0)
shapefile['COUNTYFP20'] = shapefile['COUNTYFP20'].fillna(0)
shapefile['HPAC'] = shapefile['HPAC'].fillna(0)
shapefile['RACPOP_SUM'] = shapefile['RACPOP_SUM'].fillna(0)
shapefile['ASN'] = shapefile['ASN'].fillna(0)
shapefile['AFAM'] = shapefile['AFAM'].fillna(0)

shapefile['centroid'] = shapefile.geometry.centroid
centroid_coords = np.array(
    [(point.x, point.y) for point in shapefile['centroid']] ) 
tri = Delaunay(centroid_coords) #array with indecis refering to the neighbors

#adding the neighoring relationship based on the delaunay triangulation
neighbors = {i: set() for i in range(len(centroid_coords))}
for simplex in tri.simplices:
    for i, j in zip(simplex, np.roll(simplex, 1)):
        neighbors[i].add(j)
        neighbors[j].add(i)
#add the neighbors to the shapefile
shapefile['neighbors'] = shapefile.index.map(lambda i: list(neighbors[i]))




/var/folders/_j/y8yrmv5123b_6yhhdstzl2bm0000gn/T/ipykernel_7455/2434214173.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shapefile['centroid'] = shapefile.geometry.centroid


In [105]:
shapefile[[col for col in shapefile.columns if col not in ['COUNTYFP20','VTDST','COUNTY','COUNTYFP','ASN' ,'AFAM','STATEFP20','VTDST20','RACPOP_SUM','WHT','GEOID20','NAME20','HPAC']]].columns

Index(['STATEFP', 'NAME', 'G20PREDBID', 'G20PRERTRU', 'G20PRELJOR',
       'G20PREIBLA', 'G20PREONON', 'GEOID', 'ADJPOP', 'TAWHITEALN',
       'TABLACKCMB', 'TAAIANCMB', 'TAASIANCMB', 'TANHOPICMB', 'TAOTHERALN',
       'TA2RACE', 'TAHISPANIC', 'SCALING_FA', 'AIAN', 'OTHR', 'LAT',
       'geometry', 'centroid', 'neighbors'],
      dtype='object')

In [216]:
adjacency = {i: set() for i in range(len(centroid_coords))}
for simplex in tri.simplices:
    for i, j in zip(simplex, np.roll(simplex, 1)):
        adjacency[i].add(j)
        adjacency[j].add(i)
adjacency_list = [[{"id": int(j)} for j in neighbors] for i, neighbors in adjacency.items()]


In [217]:
json_format = {
    "directed": False,
    "multigraph": False,
    "graph": [],
    "nodes": [],
    "adjacency": adjacency_list
}

In [218]:
for idx, row in shapefile.iterrows():
    node = {
        "boundary_node": False,
        "area": row.geometry.area,
        "STATEFP20": row.get('STATEFP20',''),
        "COUNTYFP20": row.get('COUNTYFP20', ''),
        "VTDST20": row.get('VTDST20',''),
        "GEOID20": row.get('GEOID20',''),
        "NAME20": row.get('NAME20',''),
        "HISP_POP": row.get('HPAC', 0),
        "WHITE_POP": row.get('WHT',0),
        "TOT_POP": row.get('RACPOP_SUM',0),
        "ASIAN_POP": row.get('ASN',0),
        "BLACK_POP": row.get('AFAM',0),
        "COUNTYFP": row.get('COUNTYFP',''),
        "COUNTY": row.get('COUNTY',''),
        "VTDST": row.get('VTDST',''),
        "id": idx
    } 
    json_format['nodes'].append(node) 


In [219]:
json_format

{'directed': False,
 'multigraph': False,
 'graph': [],
 'nodes': [{'boundary_node': False,
   'area': 0.0027047366143953,
   'STATEFP20': 32.0,
   'COUNTYFP20': 31.0,
   'VTDST20': '007549',
   'GEOID20': '32031007549',
   'NAME20': 'Precinct No. 7549',
   'HISP_POP': 0.0,
   'WHITE_POP': 0.0,
   'TOT_POP': 0.0,
   'ASIAN_POP': 0.0,
   'BLACK_POP': 0.0,
   'COUNTYFP': 31,
   'COUNTY': 'Washoe',
   'VTDST': '007549',
   'id': 0},
  {'boundary_node': False,
   'area': 0.00010358432367686621,
   'STATEFP20': 32.0,
   'COUNTYFP20': 31.0,
   'VTDST20': '008110',
   'GEOID20': '32031008110',
   'NAME20': 'Precinct No. 8110',
   'HISP_POP': 1.0,
   'WHITE_POP': 286.0,
   'TOT_POP': 357.0,
   'ASIAN_POP': 12.0,
   'BLACK_POP': 3.0,
   'COUNTYFP': 31,
   'COUNTY': 'Washoe',
   'VTDST': '008110',
   'id': 1},
  {'boundary_node': False,
   'area': 3.926884099819422e-06,
   'STATEFP20': 32.0,
   'COUNTYFP20': 31.0,
   'VTDST20': '005065',
   'GEOID20': '32031005065',
   'NAME20': 'Precinct No. 50

In [220]:
with open("NV_FORMATED.json", "w") as f:
    json.dump(json_format, f, indent=4)


In [224]:
def party_share_at_district_level(election_data, state):
    #calculate the percentage of each party at a district level
    if state == 'nv' or state == 'ut':
        election_data = election_data[election_data['district'].isin(['001','002','003','004'])]
    elif state == 'co':
        election_data = election_data[election_data['district'].isin(['001','002','003','004','005','006','007'])]
    else:
        return None
    election_data = election_data.groupby(['district', 'party_simplified']).agg({'votes': 'sum'}).reset_index()
    district_vote_totals = election_data.groupby('district')['votes'].sum().reset_index()
    election_data['%'] = election_data.apply(
        lambda x: x['votes'] / district_vote_totals[district_vote_totals['district'] == x['district']]['votes'].iloc[0] * 100, axis=1)
    election_data = election_data[election_data['party_simplified'].isin(['DEMOCRAT','REPUBLICAN'])]
    return election_data
co_election_data = pd.read_csv('Colorado/2020-co-precinct-general.csv', low_memory=False)
co_ = party_share_at_district_level(co_election_data, 'co')
co_

,district,party_simplified,votes,%
0,001,DEMOCRAT,681023,27.938832
4,001,REPUBLICAN,221607,9.091383
5,002,DEMOCRAT,362294,17.040132
9,002,REPUBLICAN,193482,9.100230
10,003,DEMOCRAT,409519,45.873123
13,003,REPUBLICAN,467228,52.337516
14,004,DEMOCRAT,249972,8.704673
18,004,REPUBLICAN,367089,12.782990
19,005,DEMOCRAT,198732,34.958055
23,005,REPUBLICAN,258216,45.421619


In [3]:
shp_path = 'nv_2020_final/nv_2020_final.shp'
shapefile = gpd.read_file(shp_path)
shapefile = shapefile.to_crs(epsg=4326)
shapefile

,STATEFP,COUNTYFP,COUNTY,VTDST,NAME,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREIBLA,G20PREONON,...,RACPOP_SUM,SCALING_FA,WHT,AFAM,AIAN,ASN,HPAC,OTHR,LAT,geometry
0,32,31,Washoe,007549,7549,0,1,0,0,0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON Z ((-120.00103 39.57869 0, -120.00148 ..."
1,32,31,Washoe,008110,8110,160,136,3,1,1,...,357.0,0.963585,286.0,3.0,11.0,12.0,1.0,9.0,23.0,"POLYGON Z ((-120.0057 39.22977 0, -120.00569 3..."
2,32,31,Washoe,005065,5065,0,0,0,0,0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON Z ((-120.0023 39.49552 0, -119.9979 39..."
3,32,31,Washoe,005053,5053,6,20,0,0,1,...,60.0,0.916667,37.0,0.0,2.0,4.0,0.0,4.0,9.0,"POLYGON Z ((-119.99007 39.51979 0, -119.98993 ..."
4,32,31,Washoe,008112,8112,35,36,3,0,3,...,131.0,0.908397,79.0,4.0,5.0,11.0,5.0,5.0,11.0,"POLYGON Z ((-119.89433 39.49135 0, -119.89392 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2089,32,31,Washoe,007582,7582,1,14,0,0,0,...,2847.0,0.872146,1484.0,166.0,59.0,171.0,31.0,164.0,409.0,"POLYGON Z ((-119.69633 39.72391 0, -119.69927 ..."
2090,32,31,Washoe,007593,7593,20,34,0,3,1,...,3051.0,0.896100,1816.0,174.0,75.0,143.0,47.0,103.0,375.0,"MULTIPOLYGON Z (((-119.64393 39.66539 0, -119...."
2091,32,31,Washoe,007547,7547,17,47,1,0,1,...,152.0,0.894737,73.0,5.0,16.0,6.0,4.0,4.0,27.0,"POLYGON Z ((-119.99908 39.90402 0, -119.99908 ..."
2092,32,31,Washoe,009125,9125,0,0,0,0,0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON Z ((-120.00608 39.37555 0, -120.00608 ..."
